<a href="https://colab.research.google.com/github/Mrcyoga10004/Deep-Learning/blob/main/Tugas_week_2_Classification_Covertdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#import libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [21]:
# Load dataset
df = pd.read_csv("/content/compressed_data.csv.gz")

In [22]:
# Hapus baris dengan NaN di target
df = df.dropna(subset=['Cover_Type'])


# Pisahkan fitur (X) dan target (y)
X = df.drop(columns=["Cover_Type"])
y = df["Cover_Type"] - 1  # Mengurangi 1 agar kelas mulai dari 0 (karena PyTorch membutuhkan label 0-based)


# Bagi dataset menjadi data latih dan uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# Normalisasi fitur
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Konversi ke Tensor PyTorch
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_train_torch = torch.tensor(y_train.values, dtype=torch.long)
y_test_torch = torch.tensor(y_test.values, dtype=torch.long)


In [23]:
# Definisi Model PyTorch
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


# Inisialisasi model
input_size = X_train.shape[1]
hidden_size = 128
num_classes = len(np.unique(y))
model_torch = MLPModel(input_size, hidden_size, num_classes)


# Definisi loss function dan optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_torch.parameters(), lr=0.001)


# Early stopping manual PyTorch
early_stop_patience = 5
best_loss = float("inf")
patience = 0

In [24]:
# Training model PyTorch
for epoch in range(200):
    optimizer.zero_grad()
    outputs = model_torch(X_train_torch)
    loss = criterion(outputs, y_train_torch)
    loss.backward()
    optimizer.step()

    # Validasi
    with torch.no_grad():
        val_outputs = model_torch(X_test_torch)
        val_loss = criterion(val_outputs, y_test_torch)

    print(f"Epoch {epoch+1}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")

    # Early stopping
    if val_loss.item() < best_loss:
        best_loss = val_loss.item()
        patience = 0
    else:
        patience += 1
    if patience >= early_stop_patience:
        print("Early stopping!")
        break

Epoch 1, Loss: 1.909448266029358, Val Loss: 1.8715571165084839
Epoch 2, Loss: 1.8710696697235107, Val Loss: 1.8342381715774536
Epoch 3, Loss: 1.8337087631225586, Val Loss: 1.7979282140731812
Epoch 4, Loss: 1.7973405122756958, Val Loss: 1.7625916004180908
Epoch 5, Loss: 1.7619340419769287, Val Loss: 1.728193998336792
Epoch 6, Loss: 1.7274572849273682, Val Loss: 1.694703459739685
Epoch 7, Loss: 1.693878173828125, Val Loss: 1.6620856523513794
Epoch 8, Loss: 1.6611658334732056, Val Loss: 1.6303082704544067
Epoch 9, Loss: 1.6292892694473267, Val Loss: 1.5993380546569824
Epoch 10, Loss: 1.59821617603302, Val Loss: 1.569146752357483
Epoch 11, Loss: 1.5679190158843994, Val Loss: 1.5397058725357056
Epoch 12, Loss: 1.5383713245391846, Val Loss: 1.5109919309616089
Epoch 13, Loss: 1.5095494985580444, Val Loss: 1.4829834699630737
Epoch 14, Loss: 1.4814329147338867, Val Loss: 1.4556621313095093
Epoch 15, Loss: 1.454002857208252, Val Loss: 1.429011344909668
Epoch 16, Loss: 1.4272434711456299, Val Los

In [25]:
# Evaluasi PyTorch
y_pred_torch = torch.argmax(model_torch(X_test_torch), axis=1).numpy()
print("Akurasi:", accuracy_score(y_test, y_pred_torch))
print("Presisi:", precision_score(y_test, y_pred_torch, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_torch, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred_torch, average='weighted'))

Akurasi: 0.7312719981411839
Presisi: 0.729260253077812
Recall: 0.7312719981411839
F1 Score: 0.7160714592426213


In [26]:
# Konversi ke TensorFlow
X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test_tf = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train, dtype=tf.int32)
y_test_tf = tf.convert_to_tensor(y_test, dtype=tf.int32)


# Definisi Model TensorFlow
model_tf = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(input_size,)),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Kompilasi model
model_tf.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Early stopping TensorFlow
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


# Training model TensorFlow
model_tf.fit(X_train_tf, y_train_tf, epochs=30, validation_data=(X_test_tf, y_test_tf), callbacks=[early_stopping])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 46s 3ms/step - accuracy: 0.7345 - loss: 0.6304 - val_accuracy: 0.7845 - val_loss: 0.5034
Epoch 2/30
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - accuracy: 0.7875 - loss: 0.4942 - val_accuracy: 0.7964 - val_loss: 0.4747
Epoch 3/30
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 77s 3ms/step - accuracy: 0.8035 - loss: 0.4621 - val_accuracy: 0.8128 - val_loss: 0.4463
Epoch 4/30
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 38s 3ms/step - accuracy: 0.8157 - loss: 0.4396 - val_accuracy: 0.8208 - val_loss: 0.4298
Epoch 5/30
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 38s 3ms/step - accuracy: 0.8225 - loss: 0.4254 - val_accuracy: 0.8290 - val_loss: 0.4186
Epoch 6/30
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - accuracy: 0.8290 - loss: 0.4125 - val_accuracy: 0.8322 - val_loss: 0.4033
Epoch 7/30
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - accuracy: 0.8321 - loss: 0.4047 - val_accuracy: 0.8293 - val_loss: 0.4069
Epoch 8/30
14526/14526 ━━━━━━━━━━━━━━━━━━━━ 78s 3ms/step - accuracy: 

In [27]:
# Evaluasi TensorFlow
y_pred_tf = np.argmax(model_tf.predict(X_test_tf), axis=1)
print("Akurasi:", accuracy_score(y_test, y_pred_tf))
print("Presisi:", precision_score(y_test, y_pred_tf, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_tf, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred_tf, average='weighted'))

3632/3632 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step
Akurasi: 0.8607264872679707
Presisi: 0.8599668241173608
Recall: 0.8607264872679707
F1 Score: 0.8596212200451879


setiap persamaan matematika Deep Learning MLP dari hasil evaluasi berikut:

Hasil PyTorch :

- Akurasi: 0.73

- Presisi: 0.72

- Recall: 0.73

- F1 Score: 0.71

Hasil TensorFlow:

- Akurasi: 0.86

- Presisi: 0.85

- Recall: 0.86

- F1 Score: 0.85

kesimpulan :

Akurasi dihitung dengan rumus akurasi, yang menunjukkan proporsi prediksi yang benar dari seluruh data. Presisi dihitung dengan prasisi, yang mengukur seberapa banyak prediksi positif yang benar dibandingkan dengan semua prediksi positif yang dibuat oleh model. Recall dihitung dengan ricall , yang mengukur seberapa baik model mengidentifikasi kelas positif dari seluruh data sebenarnya. F1 Score dihitung dengan f1 score , yang merupakan kombinasi presisi dan recall untuk memberikan gambaran lebih seimbang tentang performa model. Metrik ini membantu mengevaluasi performa model deep learning MLP secara menyeluruh.